# 新段落

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
## Imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.autograd as autograd
import matplotlib.pyplot as plt
import seaborn as sn
import numpy as np
import pandas as pd
import math

# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")
# torch.set_default_dtype(torch.float32)
torch.set_default_dtype(torch.float64)
torch.manual_seed(13)



# Sampling parameters etc
n_axis = 41
#n_axis = 21
n_time = 61
#n_time = 81

axis = torch.linspace(0,4,n_axis, device=device)
time = torch.linspace(0,12,n_time, device=device)
Ps = torch.cartesian_prod(axis,axis,time)
# Number of points
lP = Ps.shape[0]


# Initial dataset
data_axis = torch.linspace(0,4, 81, device=device)
data_time = torch.linspace(0,12,121, device=device)
data_Ps = torch.cartesian_prod(data_axis,data_axis,time)


mask = data_Ps[:,2] == 0.
X = data_Ps[mask]
dtX = data_Ps[mask]

mask1 = (abs((data_Ps[:,0]) - 4.) < 1e-10)
dxX = data_Ps[mask1]

mask2 = (abs(data_Ps[:,1] - 4.) < 1e-10)
dyX = data_Ps[mask2]

Y = torch.exp(-((X[:,0]-3)**2+(X[:,1]-3)**2)*10)
#Y = torch.exp(-(X[:,0]-0.5)**2*2.5)+torch.exp(-(X[:,1]-1)**2*5)
#Y = torch.cos((X[:,0]-1)*5)+torch.cos((X[:,1]-1)*5)
Y = Y.view(-1,1)

#dtY = -2*5*(X[:,0])*torch.exp(-(X[:,0])**2*5)-2*5*(X[:,1])*torch.exp(-(X[:,1])**2*5)
#dtY = -10*((X[:,0]-0.5)**2+(X[:,1]-0.5)**2)*torch.exp(-((X[:,0]-0.5)**2+(X[:,1]-0.5)**2)*10)
#dtY = -5*(torch.sin((X[:,0]-1)*5)+torch.sin((X[:,1]-1)*5))
#dtY = dtY.view(-1,1)

dxY = torch.where( ((dxX[:,0]-1).abs() < 1e-1) & (dxX[:,1].abs() < 1), 0., 0. )
dxY = dxY.view(-1,1)

dyY = torch.where( ((dyX[:,0]-1).abs() < 1e-1) & (dyX[:,1].abs() < 1), 0., 0. )
dyY = dyY.view(-1,1)

#dtX = dtX.to(torch.complex128)
dxX = dxX.to(torch.complex128)
dyX = dyX.to(torch.complex128)
dxY = dxY.to(torch.complex128)
dyY = dyY.to(torch.complex128)
X = X.to(torch.complex128)
Y = Y.to(torch.complex128)
#Y = torch.cat((Y,dtY,dxY,dyY),0)
Y = torch.cat((Y,dxY,dyY),0)

#sn.heatmap(Y.view(101, 101))

Using cuda device


In [ ]:
def getVarietyPoints(base):
    x,y = base.unbind(1)
    t = torch.sqrt(x.square() + y.square())

    return torch.stack([ torch.stack([x,y,t],1), torch.stack([x,y,-t],1),torch.stack([-x,y,t],1), torch.stack([-x,y,-t],1),torch.stack([x,-y,t],1), torch.stack([x,-y,-t],1) ])

def Phi(base, X):
    pts = getVarietyPoints(base)
    # return (pts.inner(X) * 1.j).exp().mean(0)
    return (pts.inner(X)).exp().mean(0)

def dtPhi(base, X):
    pts = getVarietyPoints(base)
    return ((pts.inner(X)).exp().mul(pts[:,:,2].unsqueeze(2).repeat(1, 1, pts.inner(X).shape[2])).mean(0))

def dxPhi(base, X):
    pts = getVarietyPoints(base)
    return ((pts.inner(X)).exp().mul(pts[:,:,0].unsqueeze(2).repeat(1, 1, pts.inner(X).shape[2])).mean(0))

def dyPhi(base, X):
    pts = getVarietyPoints(base)
    return ((pts.inner(X)).exp().mul(pts[:,:,1].unsqueeze(2).repeat(1, 1, pts.inner(X).shape[2])).mean(0))


def train(N):
    for epoch in range(N):
        PhiX = Phi(MC_base * 1.j, X)
        #dtPhiX = dtPhi(MC_base * 1.j, dtX)
        dxPhiX = dxPhi(MC_base * 1.j, dxX)
        dyPhiX = dyPhi(MC_base * 1.j, dyX)
        #PhiX = torch.cat((PhiX,dtPhiX,dxPhiX,dyPhiX),1)
        PhiX = torch.cat((PhiX,dxPhiX,dyPhiX),1)
        A = torch.diag_embed((eps - S_diag).exp()) + PhiX @ PhiX.H
        LA = torch.linalg.cholesky(A)
        alpha = torch.linalg.solve_triangular(LA, PhiX @ Y.to(torch.complex128), upper=False)

        nlml = 1/(2*eps.exp()) * (Y.norm().square() - alpha.norm().square())
        nlml += (PhiX.shape[1] - PhiX.shape[0])/2 * eps
        nlml += LA.diag().real.log().sum()
        nlml += 0.5*S_diag.sum()

        opt.zero_grad()
        nlml.backward()
        opt.step()

        with torch.no_grad():
            train_pred = PhiX.H @ torch.linalg.solve_triangular(LA.H, alpha, upper=True)
            err = (train_pred.real - Y).square().mean().sqrt()
            print(26*"~" + f'\nepoch {epoch}\n\
nlml {nlml}\n\
err {err}\n\
eps {eps.exp()}\n\
base std {MC_base.std(0)}\n\
min,max {train_pred.real.min().detach(),train_pred.real.max().detach()}')

In [ ]:
n_MC = 1000
# MC_axis = torch.linspace(-1,1, n_MC, device=device) * 30
MC_base = (torch.randn((n_MC, 2), device=device)).requires_grad_()
# MC_base = torch.cartesian_prod(MC_axis,MC_axis).requires_grad_()
S_diag = torch.full((n_MC,), -np.log(n_MC), requires_grad=True, device=device)
# S_diag = torch.full((n_MC**2,), -np.log(n_MC**2), requires_grad=False, device=device)
eps = torch.tensor(np.log(1e-2), requires_grad=True, device=device)


In [ ]:
opt = torch.optim.Adam([
    {'params': MC_base, 'lr': 1e-1},
    {'params': eps, 'lr': 1e-2}])
train(10000)
opt = torch.optim.Adam([
    {'params': MC_base, 'lr': 1e-1},
    {'params': eps, 'lr': 1e-2}])
train(10000)
opt = torch.optim.Adam([
    {'params': MC_base, 'lr': 1e-2},
    {'params': [S_diag, eps], 'lr': 1e-2}])
train(1000)
opt = torch.optim.Adam([
    {'params': MC_base, 'lr': 1e-3},
    {'params': [S_diag, eps], 'lr': 1e-3}])
train(300)


torch.save({
            'MC_base': MC_base.cpu(),
            'S_diag': S_diag.cpu(),
            'eps': eps.cpu(),
    }, "state.pt")

流式输出内容被截断，只能显示最后 5000 行内容。
base std tensor([6.1508, 6.1395], device='cuda:0')
min,max (tensor(-0.0077, device='cuda:0'), tensor(0.9776, device='cuda:0'))
~~~~~~~~~~~~~~~~~~~~~~~~~~
epoch 586
nlml -91710.19434746457
err (0.0015956952844077951-1.0203418542738396e-19j)
eps 3.0844508705368255e-06
base std tensor([6.1508, 6.1395], device='cuda:0')
min,max (tensor(-0.0078, device='cuda:0'), tensor(0.9776, device='cuda:0'))
~~~~~~~~~~~~~~~~~~~~~~~~~~
epoch 587
nlml -91779.0265886917
err (0.0015912339323184005-1.0136523509588244e-19j)
eps 3.0840369061725354e-06
base std tensor([6.1508, 6.1395], device='cuda:0')
min,max (tensor(-0.0076, device='cuda:0'), tensor(0.9777, device='cuda:0'))
~~~~~~~~~~~~~~~~~~~~~~~~~~
epoch 588
nlml -91804.61875468411
err (0.0015912720921976878-1.0146411691633385e-19j)
eps 3.083433821366126e-06
base std tensor([6.1508, 6.1395], device='cuda:0')
min,max (tensor(-0.0077, device='cuda:0'), tensor(0.9777, device='cuda:0'))
~~~~~~~~~~~~~~~~~~~~~~~~~~
epoch 589
nlml -9182

In [ ]:
st = torch.load("state.pt")
MC_base = st['MC_base']
S_diag = st['S_diag']
eps = st['eps']
# Prediction
#Phi_ = Phi(MC_base * 1.j, Ps.to(torch.complex128).to("cpu"))
#PhiX = Phi(MC_base * 1.j, X.to("cpu"))
#A = torch.diag_embed((eps - S_diag).exp()) + PhiX @ PhiX.H
#LA = torch.linalg.cholesky(A)
#alpha = torch.linalg.solve_triangular(LA, PhiX @ Y.to(torch.complex128).to("cpu"), upper=False)
#pred = Phi_.H @ torch.linalg.solve_triangular(LA.H, alpha, upper=True)
#pred = pred.real


#pred.detach().cpu().numpy().tofile("pred.dat")
#axis.cpu().numpy().tofile("axis.dat")
#time.cpu().numpy().tofile("time.dat")

<ipython-input-6-fda9fdf5842a>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  st = torch.load("state.pt")


In [ ]:

# Prediction
#Phi_ = Phi(MC_base * 1.j, Ps.to(torch.complex128)).to(device)
#Phi_ = Phi(MC_base1 * 1.j, MC_base2 * 1.j, Ps.to(torch.complex128).to("cpu"))
#dtPhi_ = dtPhi(MC_base1 * 1.j, MC_base2 * 1.j, Ps.to(torch.complex128).to("cpu"))
#dxPhi_ = dxPhi(MC_base1 * 1.j, MC_base2 * 1.j, Ps.to(torch.complex128).to("cpu"))
#dyPhi_ = dyPhi(MC_base1 * 1.j, MC_base2 * 1.j, Ps.to(torch.complex128).to("cpu"))
#Phi_ = torch.cat((Phi_,dtPhi_,dxPhi_,dyPhi_),1)
#Phi_ = torch.cat((Phi_,dtPhi_),1)
PhiX = Phi(MC_base * 1.j, X.to("cpu"))
#dtPhiX = dtPhi(MC_base * 1.j, dtX.to("cpu"))
dxPhiX = dxPhi(MC_base * 1.j, dxX.to("cpu"))
dyPhiX = dyPhi(MC_base * 1.j, dyX.to("cpu"))
PhiX = torch.cat((PhiX,dxPhiX,dyPhiX),1)
#PhiX = torch.cat((PhiX,dtPhiX),1)
A = torch.diag_embed((eps - S_diag).exp()) + PhiX @ PhiX.H
LA = torch.linalg.cholesky(A)
alpha = torch.linalg.solve_triangular(LA, PhiX @ Y.to("cpu").to(torch.complex128), upper=False)
Phi_ = Phi(MC_base * 1.j, Ps.to(torch.complex128).to("cpu"))
predwave = Phi_.H @ torch.linalg.solve_triangular(LA.H, alpha.to("cpu"), upper=True)
predwave = predwave.real
predwave.detach().cpu().numpy().tofile("predwave.dat")
del Phi_, predwave
dtPhi_ = dtPhi(MC_base * 1.j, Ps.to(torch.complex128).to("cpu"))
preddt = dtPhi_.H @ torch.linalg.solve_triangular(LA.H, alpha.to("cpu"), upper=True)
preddt = preddt.real
preddt.detach().cpu().numpy().tofile("preddt.dat")
del dtPhi_, preddt
dxPhi_ = dxPhi(MC_base * 1.j, Ps.to(torch.complex128).to("cpu"))
preddx = dxPhi_.H @ torch.linalg.solve_triangular(LA.H, alpha.to("cpu"), upper=True)
preddx = preddx.real
preddx.detach().cpu().numpy().tofile("preddx.dat")
del dxPhi_, preddx
dyPhi_ = dyPhi(MC_base * 1.j, Ps.to(torch.complex128).to("cpu"))
preddy = dyPhi_.H @ torch.linalg.solve_triangular(LA.H, alpha.to("cpu"), upper=True)
preddy = preddy.real
preddy.detach().cpu().numpy().tofile("preddy.dat")
del dyPhi_, preddy

#predwave.detach().cpu().numpy().tofile("predwave.dat")
#preddt.detach().cpu().numpy().tofile("preddt.dat")
#preddx.detach().cpu().numpy().tofile("preddx.dat")
#preddy.detach().cpu().numpy().tofile("preddy.dat")
axis.cpu().numpy().tofile("axis.dat")
time.cpu().numpy().tofile("time.dat")

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding
!cp predwave.dat "/content/drive/MyDrive/Colab Notebooks"
!cp preddt.dat "/content/drive/MyDrive/Colab Notebooks"
!cp preddx.dat "/content/drive/MyDrive/Colab Notebooks"
!cp preddy.dat "/content/drive/MyDrive/Colab Notebooks"
!cp axis.dat "/content/drive/MyDrive/Colab Notebooks"
!cp time.dat "/content/drive/MyDrive/Colab Notebooks"
!cp state.pt "/content/drive/MyDrive/Colab Notebooks"

In [ ]:
PhiX.shape

torch.Size([1000, 16443])